In [1]:
ID_GPU = 0

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = str(ID_GPU)

import tensorflow as tf
from keras import backend as K

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from modules.data_processing import make_Xy, make_DA, post_process, RMSLE
from modules.keras_processing import make_model

/Users/macbookpro/miniconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/macbookpro/miniconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

NB = 12  # size of cube

NUM_EPOCHS = 1000
BATCH_SIZE = 128
    
NLAYER = 3  # number of consecutive Dense layers
NUNIT = 150  # number of units in Dense layers
PDROPOUT = 0.5  # dropout probability between Dense layers

# get data from csv
X, y, _, _ = make_Xy()

# keras callbacks
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

early_stopping = EarlyStopping(monitor='loss', patience=100)  

In [3]:
### Make cross-validation

N = y.shape[0]  # number of samples 
dimX = X.shape[1]  # number of features in X

pred_CV = np.tile(np.nan, (N, 2))  # cross-validation prediction

for iCV in range(5):
    print("")
    print("---------------------")
    print("CV N°", iCV)
    print("---------------------")
    print("")
    ind_test = np.tile(False, N)
    ind_test[iCV::5] = True
        
    X_train, y_train = X[~ind_test, ], y[~ind_test, ]
    X_test, y_test = X[ind_test, ], y[ind_test, ]

    y_train = np.log(1+y_train)  # learn in log space
    y_test = np.log(1+y_test)   # learn in log space
                
    X_val, y_val, cube_val = make_DA(X_test, y=y_test, ind=ind_test)  # will be used for validation during model training
    
    model = make_model(dimX, NB, NLAYER, NUNIT, PDROPOUT)   # create keras model
        
    for iepoch in range(NUM_EPOCHS):
        X_DA, y_DA, cube_DA = make_DA(X_train, y=y_train, ind=~ind_test)  # random data-augmentation at each epoch

        model.fit([cube_DA, X_DA], y_DA, shuffle=True, verbose=True, batch_size=BATCH_SIZE, 
                   epochs=iepoch+1, initial_epoch=iepoch,
                   validation_data=[[cube_val, X_val], y_val], 
                   callbacks=[learning_rate_reduction, early_stopping])
        
    # data augmentation at test time
    pred_DA = np.zeros((10, 6*y_test.shape[0], 2))
    for iDA in range(10):
        X_DA, y_DA, cube_DA = make_DA(X_test, y=y_test, ind=ind_test)  # data-augmentation
        pred_DA[iDA, :, :] = model.predict([cube_DA, X_DA])
    
    # post process
    pred_DA = post_process(pred_DA)
        
    # average data augmentation and axis permutation
    pred_DA = pred_DA.mean(0)
    pred_DA = pred_DA.reshape((6, -1, 2))
    pred_DA = pred_DA.mean(0)
    
    # set as cross-validation prediction
    pred_CV[ind_test, :] = pred_DA

    # show competition CV score
    error0, error1 = RMSLE(y, pred_CV)
    print("")
    print("---------------------")
    print("Current RMSLE", 0.5*(error0+error1))
    print("---------------------")
    print("")



---------------------
CV N° 0
---------------------

Train on 11520 samples, validate on 2880 samples
Epoch 1/1
  640/11520 [>.............................] - ETA: 1:45:42 - loss: 1.6544

KeyboardInterrupt: 